# 3. Example: Univariate Gaussian

In [ ]:
# Install and load deps

if (!require("stringr")) {
    install.packages("stringr")
}

library(purrr)

As an example, we consider the heights in cm of 20 individuals:

We will model the heights using the univariate Gaussian. The univariate Gaussian has two
parameters, its mean and variance, which we aim to estimate.

In [ ]:
.heights_str <- "178 162 178 178 169 150 156 162 165 189 173 157 154 162 162 161 168 156 169 153"
heights <- unlist(
    map(
        str_split(.heights_str, " "),
        function(el) {
            as.numeric(el)
        }
    )
)

heights

### Step 1. Data generative model

We assume the data i.i.d. and choose the univariate Gaussian as model. The `p` will become `N` and the parameters (`mu`, `sigma^2`).

For the i.i.d., `p(X, params) = mult(p(x_i, params))


### Step 2. Simulate data

We can simulate data according to the model using the R function `rnorm()` which draws random samples of the normal distribution.

In [ ]:
simulate <- function(n, theta) {
    x <- rnorm(n, mean = theta[["mu"]], sd = theta[["sigma"]])
    return(x)
}

n <- 20
theta <- c(mu = 175, sigma = 5)

x <- simulate(n, theta)
x

### Step 3. Parameter estimation procedure

We consider maximum likelihood estimation of the parameters, so the parameters that maximize `L(theta, X)`.

Triks:
* Minimize the negative log-likelihood `NLL(theta)` which is equal to `-log(L(theta, X))`
* Reparametrize Gaussian using precision, which is `1/(sigma^2)`

The parameters estimates are gotten by setting the NLL to the null vector **0**.

*Strange math calcs.......*

`mu` is equal to the sample **mean** while `sigma` is equal to the **biased sample variance**.


### Step 4. Implementation and empirical verification

We code our estimation procedure into a R function `estimate()`

In [ ]:
estimate <- function(x) {
    mean_x <- mean(x)

    theta_hat <- c(
        mu = mean_x,
        sigma = sqrt(mean((x - mean_x)^2))
    )
    return(theta_hat)
}

In [ ]:
n <- 20
x <- simulate(n, theta = c(mu = 175, sigma = 5))

theta_hat <- estimate(x)
theta_hat

These estimates are not too far from the ground truth values. Our simple check is good enough
for this didactic toy example. The code would allow you to investigate more systematically
the relationships between estimates and ground truth with various values of the parameters
and sample size n.

### Step 5. Application to real data

We finally apply our estimation to the original dataset:

In [ ]:
theta_hat <- estimate(heights)
theta_hat

# 4. Assessing whether a distribution fits data with Q-Q plots

The strategy described in section 1 allows assessing whether an estimation procedure returns
reasonable estimates on simulated data. It does not assess however ether the simulation
assumptions, i.e. the data generative model, is a reasonable model for the data at hand.
One key modeling assumption of a data generative model is the choice of the distribution.
The quantile-quantile plot is a graphical tool to assess whether a distribution fits the data
reasonably.

As a concrete example, let’s consider 50 data points coming from the uniform distribution
in the [2,3] interval. If you assume your data comes from the uniform distribution in the
[2,3] interval, you expect the first 10% of your data to fall in [2,2.1], the second 10% in
[2.1,2.2] and so forth. A histogram could be used to visually assess this agreement. However,
histograms are shaky because of possible low counts in every bin:

In [ ]:
par(cex = 0.7)
u <- runif(50, min = 2, max = 3)
hist(u, main = "")

Instead of the histogram, one could plot the deciles of the sample distribution against those
of a theoretical distribution. Here are the deciles:

In [ ]:
dec <- quantile(u, seq(0, 1, 0.1))
dec

In [ ]:
par(cex = 0.7)
plot(seq(2, 3, 0.1),
    dec,
    xlim = c(2, 3), ylim = c(2, 3),
    xlab = "Deciles of the uniform distribution over [2,3]",
    ylab = "Deciles of the dataset"
)
abline(0, 1) ## diagonal y=x

Now we see a clear agreement between the expected values of the deciles of the theoretical
distribution (x-axis) and those empirically observed (y-axis). The advantage of this strategy is
that it also generalizes to other distributions (e.g. Normal), where the shape of the density
can be difficult to assess with a histogram.

For a finite sample we can estimate the quantile for every data point, not just the deciles.
The Q-Q plot scatter plots the quantiles of two distributions against each other. One way is
to use as expected quantile (r − 0.5)/N (Hazen, 1914), where r is the rank of the data point.
The R function ppoints gives more accurate values:


In [ ]:
par(cex = 0.7)
plot(qunif(ppoints(length(u)), min = 2, max = 3), sort(u),
    xlim = c(2, 3), ylim = c(2, 3),
    xlab = "Quantiles of the uniform distribution over [2,3]",
    ylab = "Quantiles of the dataset"
)
abline(0, 1)

In R, Q-Q plots between two datasets can be generated using the function qqplot() . In the
special case of a normal distribution use the function qqnorm() and the function qqline() ,
which adds a line to the “theoretical” quantile-quantile plot passing through the first and
third quartiles.